In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.set_option("display.max_columns",None)
pd.set_option("display.float_format",lambda x: '%.5f' % x)

df_ = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2009-2010")
df_.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [11]:
df = df_.copy()
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C",na=False)]
df = df[(df["Quantity"]>0)]
df = df[(df["Price"]>0)]
df["total_price"] = df["Quantity"] * df["Price"]

In [12]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID,total_price
count,407664.00000,407664,407664.00000,407664.00000,407664.00000
mean,13.58559,2010-07-01 10:15:11.871688192,3.29444,15368.59260,21.66491
min,1.00000,2009-12-01 07:45:00,0.00100,12346.00000,0.00100
25%,2.00000,2010-03-26 14:01:00,1.25000,13997.00000,4.95000
50%,5.00000,2010-07-09 15:47:00,1.95000,15321.00000,11.90000
75%,12.00000,2010-10-14 17:09:00,3.75000,16812.00000,19.50000
max,19152.00000,2010-12-09 20:01:00,10953.50000,18287.00000,15818.40000
std,96.84075,NaN,34.75796,1679.76214,77.15006


In [14]:
cltv_c= df.groupby("Customer ID").agg({"Invoice":lambda x:x.nunique(),
                               "Quantity":lambda x:x.sum(),
                               "total_price":lambda x:x.sum()})
cltv_c.index = cltv_c.index.astype(int)
cltv_c.columns = ["total_transaction","total_unit","total_price"];cltv_c

,total_transaction,total_unit,total_price
Customer ID,,,
12346,11,70,372.86000
12347,2,828,1323.32000
12348,1,373,222.16000
12349,3,993,2671.14000
12351,1,261,300.93000
...,...,...,...
18283,6,336,641.77000
18284,1,494,461.68000
18285,1,145,427.00000


In [15]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [18]:
repeat_rate= cltv_c[cltv_c["total_transaction"]>1].shape[0] / cltv_c.shape[0]
churn_rate = 1 - repeat_rate
churn_rate

0.32908163265306123

In [19]:
cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10

In [20]:
cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08647
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30689
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05152
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61947
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06979
...,...,...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139,64.17700,0.14883
18284,1,494,461.68000,461.68000,0.00023,46.16800,0.10707
18285,1,145,427.00000,427.00000,0.00023,42.70000,0.09903


In [21]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate ) * cltv_c["profit_margin"]
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08647,9.79736
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30689,123.40915
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05152,3.47816
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61947,502.81810
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06979,6.38188
...,...,...,...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139,64.17700,0.14883,29.02528
18284,1,494,461.68000,461.68000,0.00023,46.16800,0.10707,15.02103
18285,1,145,427.00000,427.00000,0.00023,42.70000,0.09903,12.84912


In [35]:
cltv_c.sort_values(by="cltv",ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102,89,124216,349164.35000,3923.19494,0.02064,34916.43500,80.97503,8591666.19527
14646,78,170278,248396.50000,3184.57051,0.01809,24839.65000,57.60587,4348190.36027
14156,102,108107,196566.74000,1927.12490,0.02365,19656.67400,45.58598,2722937.51052
14911,205,69722,152147.57000,742.18327,0.04754,15214.75700,35.28469,1631351.87152
13694,94,125893,131443.19000,1398.33181,0.02180,13144.31900,30.48311,1217569.56993
...,...,...,...,...,...,...,...,...
18115,1,3,9.70000,9.70000,0.00023,0.97000,0.00225,0.00663
15040,1,1,7.49000,7.49000,0.00023,0.74900,0.00174,0.00395
15913,1,3,6.30000,6.30000,0.00023,0.63000,0.00146,0.00280


In [38]:
cltv_c["segment"] = pd.qcut(cltv_c["cltv"],4,labels=["D","C","B","A"])
cltv_c.groupby("segment").agg(["count","mean"])

total_transaction          total_unit            total_price  \
                    count     mean      count       mean       count   
segment                                                                
D                    1078  1.22913       1078  109.20686        1078   
C                    1078  2.00649       1078  283.47217        1078   
B                    1078  3.76809       1078  680.71614        1078   
A                    1078 10.81911       1078 4064.22449        1078   

                   average_order_value           purchase_frequency          \
              mean               count      mean              count    mean   
segment                                                                       
D        178.63922                1078 157.80465               1078 0.00029   
C        476.26545                1078 294.33285               1078 0.00047   
B       1132.30296                1078 390.09746               1078 0.00087   
A       6405.74532                1078 671.05554               1078 0.00251   

        profit_margin           customer_value          cltv              
                count      mean          count    mean count        mean  
segment                                                                   
D                1078  17.86392           1078 0.04143  1078     2.65309  
C                1078  47.62654           1078 0.11045  1078    16.91865  
B                1078 113.23030           1078 0.26259  1078    96.35431  
A                1078 640.57453           1078 1.48556  1078 23462.60162

In [39]:
df = df_.copy()

def cltv_c(df,profit=0.10,csv=False):
    
    #data prep
    df.dropna(inplace=True)
    df = df[~df["Invoice"].str.contains("C",na=False)]
    df = df[(df["Quantity"]>0)]
    df = df[(df["Price"]>0)]
    df["total_price"] = df["Quantity"] * df["Price"]
    cltv_c = df.groupby("Customer ID").agg({"Invoice":lambda x:x.nunique(),
                                            "Quantity":lambda x:x.sum(),
                                            "total_price":lambda x:x.sum()})
    cltv_c.index = cltv_c.index.astype(int)
    cltv_c.columns = ["total_transaction","total_unit","total_price"]  
    
    #avg_order_value
    cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]  
    
    #purchase_frequency
    cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]
    
    #repeat-churn_rate
    repeat_rate = cltv_c[cltv_c["total_transaction"]>1].shape[0] / cltv_c.shape[0]
    churn_rate = 1 - repeat_rate
    
    #profit_margin
    cltv_c["profit_margin"] = cltv_c["total_price"] * profit
    
    #customer_value
    cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]
    
    #customer_lifetime_value
    
    cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]
    
    #segment
    cltv_c["segment"] = pd.qcut(cltv_c["cltv"],4,labels=["D","C","B","A"])
    
    if csv:
        cltv_c.to_csv("cltv_c.csv")
    
    
    return cltv_c

In [40]:
cltv_c(df)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346,11,70,372.86000,33.89636,0.00255,37.28600,0.08647,9.79736,C
12347,2,828,1323.32000,661.66000,0.00046,132.33200,0.30689,123.40915,B
12348,1,373,222.16000,222.16000,0.00023,22.21600,0.05152,3.47816,D
12349,3,993,2671.14000,890.38000,0.00070,267.11400,0.61947,502.81810,A
12351,1,261,300.93000,300.93000,0.00023,30.09300,0.06979,6.38188,D
...,...,...,...,...,...,...,...,...,...
18283,6,336,641.77000,106.96167,0.00139,64.17700,0.14883,29.02528,C
18284,1,494,461.68000,461.68000,0.00023,46.16800,0.10707,15.02103,C
18285,1,145,427.00000,427.00000,0.00023,42.70000,0.09903,12.84912,C
